In [25]:
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
dataset = pd.read_csv('../data/preprocessed_data.csv',encoding='ISO-8859-1')
dataset = dataset.drop(['Unnamed: 0'], axis=1)
dataset = dataset.dropna(subset=['Text'])
# dataset = dataset.sample(frac=0.85, random_state=42)
dataset

Text    Source  Human
0       Federal law supersedes state law cannabis medi...  Bloom-7B      0
1       Miles feels restless working day decides go pa...  Bloom-7B      0
2       first danish means follow american politics mu...  Bloom-7B      0
3       paper present novel rulebased approach Runtime...  Bloom-7B      0
4       social progressives love democracy relatively ...  Bloom-7B      0
...                                                   ...       ...    ...
788917  vast expanse time echoes aeons mingle whispers...    YI-34B      0
788918  phenomenon brain drain particularly STEM field...    YI-34B      0
788919  Influence Climate Change Marine Ecosystems Cli...    YI-34B      0
788920  Title Case Limiting Car Usage Navigating Towar...    YI-34B      0
788921  vast expanse globalized society cultures inter...    YI-34B      0

[788920 rows x 3 columns]

In [27]:
# Load the pre-fitted vectorizer
tfidf = TfidfVectorizer(max_features=7000)
X = tfidf.fit_transform(dataset['Text'])

In [28]:
params = {
        'penalty': ['l1', 'l2'],
        'C': [0.1, 1, 10],
        'solver': ['liblinear', 'saga']
    }

In [29]:
y = dataset['Human']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [30]:
log = LogisticRegression()

In [31]:
# random_search = RandomizedSearchCV(log, param_distributions=params, n_iter=50, cv=3, random_state=42)

In [32]:
grid_search = GridSearchCV(log, params, n_jobs=4, verbose=10, cv=5)
# log.set_params(C=1.7, max_iter=150, penalty='l1', solver='saga')

In [33]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


In [ ]:
print(f"Best Hyperparameters: {grid_search.best_params_}")

Best Hyperparameters: {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}


In [ ]:
log.set_params(C=1, penalty='l1', solver='liblinear')

LogisticRegression(C=1, penalty='l1', solver='liblinear')

In [ ]:
log.fit(X_train, y_train)

LogisticRegression(C=1, penalty='l1', solver='liblinear')

In [ ]:
# predictions
preds_log = log.predict(X_test)
preds_log

array([1, 1, 0, ..., 0, 0, 0])

In [ ]:
# accuracy scores
print(f"Train Score: {log.score(X_train, y_train)}")
print(f"Test Score: {log.score(X_test, y_test)}")
#print(f"Logistic Regression Accuracy Score: {accuracy_score(preds_log, y_test)}")

Train Score: 0.8460915827061951
Test Score: 0.8421470702563842


In [ ]:
joblib.dump(log, '../models/grid_logistic_regression_model.pkl')

['../models/grid_logistic_regression_model.pkl']

In [ ]:
type(X_test)

scipy.sparse._csr.csr_matrix